nest_asyncio는 Python의 asyncio 라이브러리에서 이벤트 루프를 중첩해서 사용할 수 있도록 도와주는 라이브러리입니다.

기본적으로 Python의 asyncio는 한 번 실행된 이벤트 루프를 다시 실행하려고 하면 오류를 발생시킵니다. 예를 들어, Jupyter Notebook 같은 환경에서는 이미 실행 중인 이벤트 루프가 있기 때문에, 새로운 비동기 작업을 실행하려고 하면 문제가 생길 수 있습니다.

nest_asyncio는 이런 문제를 해결하기 위해 기존 이벤트 루프를 패치하여, 이미 실행 중인 이벤트 루프에서도 추가적인 비동기 작업을 실행할 수 있도록 만들어줍니다.

간단한 비유:
기본 asyncio: "이미 도로에 차가 달리고 있으니, 새로운 차를 추가로 못 올려!"
nest_asyncio: "도로를 확장해서 새로운 차도 추가로 달릴 수 있게 만들었어!"

In [13]:
import nest_asyncio
nest_asyncio.apply()

In [15]:

import uvicorn
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def hello():
    return {"message": "Hello World"}

uvicorn.run(app)

INFO:     Started server process [20589]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56097 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56097 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56097 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56097 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56099 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:56102 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20589]


# Database 설정 파일 추가

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./myapi.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

이 코드는 Python의 SQLAlchemy 라이브러리를 사용하여 데이터베이스와 상호작용하기 위한 설정을 정의합니다. SQLAlchemy는 Python에서 데이터베이스 작업을 쉽게 처리할 수 있도록 도와주는 강력한 ORM(Object Relational Mapper) 도구입니다. 아래는 코드의 각 부분에 대한 설명입니다.

---

1. **필요한 모듈 임포트**  
   첫 번째로, `sqlalchemy` 라이브러리에서 필요한 구성 요소를 가져옵니다.  
   - `create_engine`: 데이터베이스와의 연결을 생성하는 데 사용됩니다.  
   - `declarative_base`: ORM에서 데이터베이스 테이블을 Python 클래스와 매핑하기 위한 기본 클래스입니다.  
   - `sessionmaker`: 데이터베이스 세션을 생성하고 관리하는 데 사용됩니다.

---

2. **데이터베이스 URL 설정**  
   `SQLALCHEMY_DATABASE_URL` 변수는 데이터베이스 연결 문자열을 정의합니다. 여기서는 SQLite 데이터베이스를 사용하며, 파일 이름은 `myapi.db`입니다. SQLite는 파일 기반 데이터베이스로, 간단한 프로젝트에 적합합니다.

---

3. **데이터베이스 엔진 생성**  
   `create_engine` 함수는 데이터베이스와의 연결을 처리하는 엔진 객체를 생성합니다.  
   - `SQLALCHEMY_DATABASE_URL`을 사용하여 데이터베이스 위치를 지정합니다.  
   - `connect_args={"check_same_thread": False}`는 SQLite에서 멀티스레드 환경에서도 동일한 연결을 사용할 수 있도록 설정합니다. 이는 SQLite의 기본 동작을 변경하는 옵션입니다.

---

4. **세션 로컬 생성**  
   `SessionLocal`은 데이터베이스 세션을 생성하는 팩토리 함수입니다.  
   - `autocommit=False`: 세션에서 변경 사항을 명시적으로 커밋해야 합니다.  
   - `autoflush=False`: 세션이 자동으로 변경 사항을 데이터베이스에 반영하지 않습니다.  
   - `bind=engine`: 이 세션이 위에서 생성한 데이터베이스 엔진과 연결되도록 설정합니다.

---

5. **기본 클래스 정의**  
   `Base = declarative_base()`는 ORM에서 사용할 기본 클래스를 생성합니다. 이 클래스는 이후 데이터베이스 테이블과 매핑될 모든 모델 클래스의 부모 클래스 역할을 합니다.

---

이 코드는 데이터베이스와 상호작용하기 위한 기본 설정을 정의하며, 이후 데이터베이스 모델을 생성하거나 데이터를 삽입/조회하는 데 사용됩니다. 이 구조는 SQLAlchemy를 사용하는 대부분의 프로젝트에서 공통적으로 사용되는 초기 설정입니다.

In [ ]:
from sqlalchemy import Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import relationship

class Question(Base):
    __tablename__ = "questions"

    id = Column(Integer, primary_key=True)
    subject = Column(String, nullable=False)
    content = Column(Text, nullable=False)
    create_date = Column(DateTime, nullable=False)

class Answer(Base):
    __tablename__ = "answers"

    id = Column(Integer, primary_key=True)
    content = Column(Text, nullable=False)
    create_date = Column(DateTime, nullable=False)
    question_id = Column(Integer, ForeignKey("questions.id"))
    question = relationship("Question", backref="answers")
    

위의 코드는 SQLAlchemy를 사용하여 데이터베이스 테이블과 열(column), 그리고 외래 키(foreign key)를 정의하고 관리하는 데 필요한 클래스와 함수들을 포함하고 있습니다. SQLAlchemy는 Python에서 데이터베이스와 상호작용하기 위한 강력한 ORM(Object Relational Mapping) 라이브러리로, 이 코드는 데이터베이스 테이블의 스키마를 정의하고 관계를 설정하는 데 사용됩니다.

### `Answer` 클래스
`Answer` 클래스는 SQLAlchemy의 ORM을 사용하여 데이터베이스 테이블을 정의한 예입니다. 이 클래스는 `Base` 클래스를 상속받으며, `__tablename__` 속성을 통해 데이터베이스 테이블 이름을 `"answers"`로 지정합니다. 이 클래스는 다음과 같은 열을 정의합니다:
- `id`: 기본 키로 사용되는 정수형 열.
- `content`: 텍스트 데이터를 저장하며, `nullable=False`로 설정되어 있어 값이 반드시 있어야 합니다.
- `create_date`: 답변이 생성된 날짜와 시간을 저장하는 `DateTime` 열.
- `question_id`: 외래 키로, `"questions"` 테이블의 `id` 열을 참조합니다.
- `question`: `relationship`을 통해 `Question` 클래스와의 관계를 설정하며, `backref`를 사용하여 `answers`라는 역참조를 제공합니다.

### `relationship` 함수
`relationship` 함수는 SQLAlchemy에서 두 테이블 간의 관계를 정의하는 데 사용됩니다. 이 함수는 부모-자식 관계 또는 다대다 관계를 설정할 수 있습니다. 예를 들어, `Answer` 클래스에서 `question` 속성은 `Question` 클래스와의 관계를 나타냅니다. `backref` 매개변수를 통해 `Question` 클래스에서 `answers`라는 속성을 자동으로 생성하여 양방향 관계를 설정합니다. 이 함수는 다양한 매개변수를 지원하며, 관계의 로딩 전략(`lazy`), 삭제 시 동작(`cascade`), 정렬 순서(`order_by`) 등을 세부적으로 제어할 수 있습니다.

### `Column` 클래스
`Column` 클래스는 데이터베이스 테이블의 열을 나타냅니다. 이 클래스는 열의 이름, 데이터 타입, 기본값, 고유성, 인덱스 여부 등을 정의할 수 있습니다. 예를 들어, `id` 열은 `primary_key=True`로 설정되어 기본 키로 지정되며, `content` 열은 `nullable=False`로 설정되어 값이 반드시 필요합니다. 또한, `ForeignKey` 객체를 사용하여 다른 테이블의 열을 참조할 수 있습니다.

### `ForeignKey` 클래스
`ForeignKey` 클래스는 열 간의 외래 키 관계를 정의합니다. 예를 들어, `question_id` 열은 `ForeignKey("questions.id")`를 통해 `"questions"` 테이블의 `id` 열을 참조합니다. 이 클래스는 외래 키 제약 조건을 설정하며, `ondelete`, `onupdate`와 같은 매개변수를 통해 삭제 및 업데이트 시의 동작을 제어할 수 있습니다.

### 주요 동작
이 코드의 주요 동작은 데이터베이스 테이블과 열을 정의하고, 테이블 간의 관계를 설정하며, 외래 키를 통해 참조 무결성을 유지하는 것입니다. SQLAlchemy는 이러한 정의를 기반으로 데이터베이스 스키마를 생성하거나, ORM을 통해 Python 객체와 데이터베이스 간의 매핑을 제공합니다. 이를 통해 개발자는 SQL 쿼리를 직접 작성하지 않고도 데이터베이스와 상호작용할 수 있습니다.

# alembic
Alembic은 Python의 데이터베이스 마이그레이션 도구로, SQLAlchemy와 함께 사용하여 데이터베이스 스키마를 관리하고 버전 관리를 수행할 수 있습니다.

아래 파일들을 수정합니다.

[alembic.ini]
sqlalchemy.url = sqlite:///./myapi.db

[migrations/env.py]
import models

target_metadata = models.Base.metadata

### 마이그레이션 파일 생성
새로운 데이터베이스 변경 사항(예: 테이블 추가, 수정)을 반영하기 위해 마이그레이션 파일을 생성합니다.

--autogenerate 옵션은 SQLAlchemy의 Base.metadata를 기반으로 변경 사항을 자동으로 감지합니다.
생성된 파일은 versions/ 디렉토리에 저장됩니다.

alembic revision --autogenerate

### 리비전 파일 실행

alembic upgrade head

In [16]:
from models import Question, Answer
from datetime import datetime
from database import SessionLocal

db = SessionLocal()

In [17]:
# 데이터 넣기
q = Question(subject='pybo가 무엇인가요?', content='pybo에 대해서 알고 싶습니다.', create_date=datetime.now())
db.add(q)
db.commit()

q.id

1

In [18]:
q = Question(subject='FastAPI 모델 질문입니다.', content='id는 자동으로 생성되나요?', create_date=datetime.now())
db.add(q)
db.commit()

q.id

2

In [19]:
# 데이터 조회하기
db.query(Question).all()

[<models.Question at 0x10e60a450>, <models.Question at 0x10da1ac40>]

In [20]:
# filter 함수 이용한 질문
db.query(Question).filter(Question.id == 1).all()

In [21]:
# id는 유일한 값이므로 get 함수를 이용해 조회할 수도 있음
db.get(Question, 1)

In [22]:
# Like 검색
db.query(Question).filter(Question.subject.like('%FastAPI%')).all()

- 데이터를 조회하는 다양한 사용법
- https://docs.sqlalchemy.org/en/14/orm/query.html

In [23]:
# 데이터 수정하기
q = db.get(Question, 2)
q.subject = 'FastAPI Model Question'
db.commit()


In [24]:
# 데이터 삭제하기
q = db.get(Question, 1)
db.delete(q)
db.commit()

In [25]:
db.query(Question).all()

In [26]:
# 답변 데이터 저장하기
q = db.get(Question, 2)
a = Answer(question=q, content='네 자동으로 생성됩니다.', create_date=datetime.now())
db.add(a)
db.commit()

In [27]:
a.id

1

In [28]:
a.question

In [29]:
q.answers